In [1]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()
import os

In [4]:
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

def create_chains(prompt_template):
    prompt=PromptTemplate.from_template(prompt_template)
    return prompt | llm

# Direct Task Specification

In [5]:
direct_prompt_task=""""
        Classify the sentiment of the following text as positive, negative or neutral.
        Do not explain your reasoning, just provide the classification
        Text: {text}
        Sentiment:
                    """

direct_task_chain=create_chains(direct_prompt_task)

texts=[
    "I absolutely loved the movie! The acting was superb.",
    "The weather today is quite typical for this time of year.",
    "I'm disappointed with the service I received at the restaurant."
]

for text in texts:
    result=direct_task_chain.invoke({'text':text}).content
    print(f"Text:{text}")
    print(f'Sentiment:{result}')

Text:I absolutely loved the movie! The acting was superb.
Sentiment:Positive
Text:The weather today is quite typical for this time of year.
Sentiment:Neutral
Text:I'm disappointed with the service I received at the restaurant.
Sentiment:Negative


# Format Specification

In [7]:
format_specific_prompt="""
        Generate a short news article about {topic}.
        Structure your response in the following format:
        Headline:[A catcht headline for the article]
        Lead:[A brief introductory paragraph summarizing the key points]
        Body:[2-3 paragrapsh providing more details]
        conclusion:[A concluding sentence or call to action]
                        """
format_specific_chain=create_chains(format_specific_prompt)

# Test the format specification prompting
topic='The discovery of Black Hole'
result=format_specific_chain.invoke({'topic':topic}).content
print(result)

**Headline:**  Giant Black Hole Defies Expectations, Rewriting Cosmic History

**Lead:**  Astronomers have announced the discovery of a supermassive black hole so unexpectedly large and ancient that it challenges existing models of galaxy formation.  Located at the heart of a distant quasar, the behemoth weighs in at a staggering 30 billion times the mass of our sun, pushing the boundaries of what scientists thought possible in the early universe.


**Body:** The black hole, designated J0100+2802, was identified through observations using a network of telescopes, including the Very Large Telescope in Chile. Its immense size is particularly surprising given its relatively early appearance in the universe's history – just 900 million years after the Big Bang.  Current theories predict that black holes of this scale should take far longer to form.  The discovery suggests that the early universe may have been far more conducive to the rapid growth of supermassive black holes than previousl

In [9]:
multi_step_prompt = """Analyze the following text for its main argument, supporting evidence, and potential counterarguments. 
Provide your analysis in the following steps:

1. Main Argument: Identify and state the primary claim or thesis.
2. Supporting Evidence: List the key points or evidence used to support the main argument.
3. Potential Counterarguments: Suggest possible objections or alternative viewpoints to the main argument.

Text: {text}

Analysis:"""

multi_step_chain = create_chains(multi_step_prompt)

# Test the multi-step reasoning approach
text = """While electric vehicles are often touted as a solution to climate change, their environmental impact is not as straightforward as it seems. 
The production of batteries for electric cars requires significant mining operations, which can lead to habitat destruction and water pollution. 
Moreover, if the electricity used to charge these vehicles comes from fossil fuel sources, the overall carbon footprint may not be significantly reduced. 
However, as renewable energy sources become more prevalent and battery technology improves, electric vehicles could indeed play a crucial role in combating climate change."""

result = multi_step_chain.invoke({"text": text}).content
print(result)

**Analysis:**

**1. Main Argument:** The main argument is that the environmental impact of electric vehicles (EVs) is more complex than often portrayed, and their contribution to climate change mitigation is contingent upon factors like battery production methods and electricity source.  It's not a simple case of EVs being a straightforward solution.


**2. Supporting Evidence:**

* **Resource-intensive battery production:** The text highlights the environmental damage caused by mining operations necessary for EV battery production, including habitat destruction and water pollution.
* **Electricity source dependence:** The text points out that if EVs are charged using electricity generated from fossil fuels, their carbon footprint might not be significantly reduced, negating a key benefit.


**3. Potential Counterarguments:**

* **Lifecycle assessments show overall emissions reduction:**  Studies comparing the entire lifecycle emissions of EVs versus gasoline cars might show a net redu

In [11]:
def compare_prompts(task, prompt_templates):
    """
    Compare different prompt templates for the same task.
    
    Args:
        task (str): The task description or input.
        prompt_templates (dict): A dictionary of prompt templates with their names as keys.
    """
    print(f"Task: {task}\n")
    for name, template in prompt_templates.items():
        chain = create_chains(template)
        result = chain.invoke({"task": task}).content
        print(f"{name} Prompt Result:")
        print(result)
        print("\n" + "-"*50 + "\n")

task = "Explain conciesly the concept of blockchain technology"

prompt_templates = {
    "Basic": "Explain {task}.",
    "Structured": """Explain {task} by addressing the following points:
1. Definition
2. Key features
3. Real-world applications
4. Potential impact on industries"""
}

compare_prompts(task, prompt_templates)

Task: Explain conciesly the concept of blockchain technology

Basic Prompt Result:
Blockchain is a shared, immutable ledger recording transactions in "blocks" linked cryptographically.  This creates a transparent, secure, and decentralized system resistant to tampering.

--------------------------------------------------

Structured Prompt Result:
1. **Definition:** Blockchain is a decentralized, immutable ledger shared across a network of computers.  It records transactions in "blocks" linked cryptographically, creating a tamper-proof chain.

2. **Key Features:** Decentralization (no single point of control), immutability (records cannot be altered), transparency (transactions are viewable), security (cryptography ensures integrity).

3. **Real-world applications:** Cryptocurrencies (Bitcoin), supply chain management (tracking goods), digital identity (secure verification), voting systems (transparent and secure elections).

4. **Potential impact on industries:** Increased transparenc